# Transformers Model From Scratch

**References**
- [Coding a Transformer from scratch on PyTorch, with full explanation, training and inference](https://youtu.be/ISNdQcPhsts?si=M80AnG5chc6sKzk5)
- *[An Introduction to Transformers](https://arxiv.org/abs/2304.10557)*



In [1]:
# Imports
import torch
import torch.nn as nn

import math

## Input Embeddings

![Input Embeddings](images/InputEmbeddings.png)

*Why are we multiplying the embedding output by the square root of the dimension?*


In [2]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None:
        """Embedding Module.
        The output embedding is multiplied by the square root of the dimension.
        
        Args:
            d_model (int): Dimension of Input Embedding
            vocab_size (int): Vocabulary Size
        """
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)


## Positional Encoding

![Positional Embedding](images/PositionalEmbedding.png)

The formula used to create the positional encoding:
![Positional Embedding Vector](images/PositionalEmbeddingVector.png)

*Why is there an odd and even position equation?*



In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        """_summary_

        Args:
            d_model (int): Size of the vector the positional encoder should be.
            seq_len (int): The maximum length of the sentence since we need to create one vector for each position.
            dropout (float): Make the model less overfit.
        """
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        # Create a matrix of shape(seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len, 1)
        # torch.arange - returns a 1D tensor of size (end-start/step) with values from the interval [start, end) 
        # taken with common difference step beginning from start.
        # ? Why are we unsqueezing here?
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        # ? Formula - I am not sure of the arange part here and how it translates to the equation above
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # ? It seems that you are doing this log this for numerical stability, need to look into why?
        # Apply the sin to even position
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # Adding a new dimension to account for the batch of sentence, we use unsqueeze to do this.
        pe = pe.unsqueeze(0)  # (1, seq_len, d_model)

        # register the po tensor to the buffer 
        # Why: to keep inside the module, not as a learned param, but to be saved along with the state of the model
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # Since we are not learning this tensor, we add requires grad as False
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad(False)
        return self.dropout(x)


## Layer Normalization

![Layer Normalization](images/LayerNormalization.png)




In [4]:
class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps: float = 10**-6) -> None:
        """_summary_

        Args:
            features (int): _description_
            eps (float, optional): Needed to prevent div by 0 error. Defaults to 10**-6.
        """
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features))  # Multiplied
        self.bias = nn.Parameter(torch.zeros(features))  # Added
    
    def forward(self, x):
        mean = x.mean(div=-1, keep_dim=True)
        std = x.std(dim=-1, keep_dim=True)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias
    

## Feed Forward



In [ ]:
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        """_summary_

        Args:
            d_model (int): _description_
            d_ff (int): _description_
            dropout (float): _description_
        """
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)  # W1 and B1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)  # W2 and B2
    
    def forward(self, x):
        # (Batch, seq_len, d_model) -> (Batch, seq_len, d_ff) -> (Batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))


## Multi-Head Attention

![Multi-Head Attention](images/MultiHeadAttention.png)


In [ ]:
class MultiHeadAttentionBlock(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        """_summary_

        Args:
            d_model (int): _description_
            h (int): _description_
            dropout (float): _description_
        """
        super().__init__()
        self.d_model = d_model
        self.h = h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h  # ? Why // instead of just /

        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    # staticmethod: because we can call this without an instance of the above class
    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]

        # (Batch, h, seq_len, d_k) -> (Batch, h, seq_len, seq_len)
        # @ matrix multiplication in pytorch
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        # masking is just multiplying by a very small value which then becomes zero after applying softmax
        if mask is not None:
            # replace all the values for mask==0 with -1e9
            attention_scores.masked_fill(mask==0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1)  # (Batch, h, seq_len, seq_len)

        if dropout is not None:
            attention_scores = dropout(attention_scores)
        
        # The second value in the tuple is used for visualization
        return (attention_scores @ value), attention_scores

    
    def forward(self, q, k, v, mask):
        query = self.w_q(q)  # (Batch, seq_len, d_model) -> (Batch, seq_len, d_model)
        key = self.w_k(k)  # (Batch, seq_len, d_model) -> (Batch, seq_len, d_model)
        value = self.w_v(v)  # (Batch, seq_len, d_model) -> (Batch, seq_len, d_model)
    
        # (Batch, seq_len, d_model) -> (Batch, seq_len, h, d_k) -> (Batch, h, seq_len, d_k)
        # Using view: keep batch dim since we don't want to split the sentence, we want to split the embedding
        # torch.tensor.view: returns a new tensor with the same data as the self tensor but of a different shape.
        # Why transpose: prefer h as the second dim, this way each head will see all the sentences
        # ? Why use view instead of reshape
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Getting attention scores
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # Concatenating the attention scores
        # (Batch, h, seq_len, d_k) -> (Batch, seq_len, h, d_k) -> (Batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)  
        return self.w_o(x)



## Residual Connection

In [ ]:
class ResidualConnection(nn.Module):
    def __init__(self, features: int, dropout: float) -> None:
        """_summary_

        Args:
            features (int): _description_
            dropout (float): _description_
        """
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)
    
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

## Encoder Block



In [ ]:
class EncoderBlock(nn.Module):
    def __init__(
            self,
            features: int,
            self_attention_block: MultiHeadAttentionBlock,
            feed_forward_block: FeedForwardBlock,
            dropout: float
        ) -> None:
        """_summary_

        Args:
            self_attention_block (MultiHeadAttentionBlock): _description_
            feed_forward_block (FeedForwardBlock): _description_
            dropout (float): _description_
        """
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connection = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])
    
    def forward(self, x, src_mask):
        x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connection[1](x, self.feed_forward_block)
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, features:int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)
    
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

## Decoder Block